<a href="https://colab.research.google.com/github/kretchmar/CS339_2023/blob/main/HiHoCherryO_DP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving Hi Ho Cherry O Game using Linear Algebra

## Game Description

In the game, each player has a bucket.  The goal is to put 10 cherries in your own bucket to win the game.  Each player starts with zero cherries.  On each turn, a player spins a spinner.  The spinner has 7 outcomes

- Add 1 cherry
- Add 2 cherries
- Add 3 cherries
- Add 4 cherries
- Remove 2 cherries (bird eats them)
- Remove 2 cherries (dog eats them)
- Remove all cherries (spill the bucket)

Our goal is to figure out how many spins it takes on average to reach a full bucket of 10 cherries.


## Equations

We let $s_i$ represent the value of being in state $i$; that is, with $i$ cherries in the bucket.   The value of state $s_i$ is the expected number of spins needed to reach 10 cherries when starting with $i$ cherries.  So $s_0$ will be the expected number of spins when starting with 0 cherries -- the start state of the game and the answer that we seek.

The values for $s_i$ are circular in reference.  They refer to each other because there are probabilistic chances of ending up in different states.  For example, from state 6 ($s_6$), we have a 1/7 chance of transitioning to state $s_7$ (1 cherry), $s_8$ (2 cherries), $s_9$ (3 cherries), $s_{10}$ (+4 cherries and the goal), $s_0$ (spill bucket) and a 2/7 chance of transitioning to $s_4$ (-2 cherries for dog or bird).   This results in equation

$
\begin{eqnarray*}
s_6 & = 1 + & \frac{1}{7} s_7 + & \frac{1}{7} s_8 + & \frac{1}{7} s_9 + & \frac{1}{7} s_{10} + & \frac{1}{7} s_0 + & \frac{2}{7} s_4
\end{eqnarray*}
$

where the 1 comes from the spin we are doing now.   This creates 11 equations:

$
\begin{eqnarray*}
s_0 & = 1 + & \frac{3}{7} s_0 + & \frac{1}{7} s_1 + & \frac{1}{7} s_2 + & \frac{1}{7} s_{3} + & \frac{1}{7} s_4 \\
s_1 & = 1 + & \frac{3}{7} s_0 + & \frac{1}{7} s_2 + & \frac{1}{7} s_3 + & \frac{1}{7} s_4 + & \frac{1}{7} s_5 \\
s_2 & = 1 + & \frac{3}{7} s_0 + & \frac{1}{7} s_3 + & \frac{1}{7} s_4 + & \frac{1}{7} s_5 + & \frac{1}{7} s_6 \\
s_3 & = 1 + & \frac{1}{7} s_0 + & \frac{2}{7} s_1 + & \frac{1}{7} s_4 + & \frac{1}{7} s_5 + & \frac{1}{7} s_6 + & \frac{1}{7} s_7 \\
... \\
s_9 & = 1 + & \frac{1}{7} s_0 + & \frac{2}{7} s_7 + & \frac{4}{7} s_{10} \\
s_{10} & = 0
\end{eqnarray*}
$

We want to move all the coefficients to the RHS and the non-coefficient terms to the LHS.  

$
\begin{eqnarray*}
1  & =  \frac{4}{7} s_0 - & \frac{1}{7} s_1 - & \frac{1}{7} s_2 - & \frac{1}{7} s_3 - & \frac{1}{7} s_4 \\
1  & =  \frac{4}{7} s_0 - & \frac{1}{7} s_2 - & \frac{1}{7} s_3 - & \frac{1}{7} s_4 - & \frac{1}{7} s_5 \\
... \\
0 & = s_{10}
\end{eqnarray*}
$

Now this is in the form of $b = Ax$ where $x$ is the vector of unknowns:

$
\begin{eqnarray*}
x = [ s_0, s_1, s_2, ..., s_{10}]^T
\end{eqnarray*}
$

and $b$ is our column vector of non-coefficient terms

$
\begin{eqnarray*}
b = [ 1, 1, 1, ..., 1, 0]^T
\end{eqnarray*}
$

and $A$ is the matrix of coefficients:

$
\begin{eqnarray*}
A = \left[   
\begin{array}{rrrrrrrrrrr}
\frac{4}{7} & -\frac{1}{7}  & -\frac{1}{7}  & -\frac{1}{7}  & -\frac{1}{7} & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
-\frac{3}{7} & 1 & -\frac{1}{7}  & -\frac{1}{7}  & -\frac{1}{7}  & -\frac{1}{7} & 0 & 0 & 0 & 0 & 0 & 0  \\
-\frac{3}{7} & 0 & 1 &  -\frac{1}{7}  & -\frac{1}{7}  & -\frac{1}{7}  & -\frac{1}{7} & 0 & 0 & 0 & 0 & 0  \\
-\frac{1}{7} & -\frac{2}{7} & 0 & 1 &  -\frac{1}{7}  & -\frac{1}{7}  & -\frac{1}{7}  & -\frac{1}{7} & 0 & 0 & 0  & 0 \\
... \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
\end{array} \right]
\end{eqnarray*}
$



In [1]:
import numpy as np

In [2]:
# Note to reader: In the analysis below, I assumed the dog and bird spinner would remove 3 cherries (not 2)
# so the equations and results are slightly different, though the process is the same.  

A = np.zeros((11,11))
A[0] = [4/7, -1/7, -1/7, -1/7, -1/7, 0, 0,0,0,0,0]
A[1] = [-3/7,1,-1/7, -1/7, -1/7, -1/7, 0, 0,0,0,0]
A[2] = [-3/7,0,1,-1/7, -1/7, -1/7, -1/7, 0, 0,0,0]
A[3] = [-3/7,0,0,1,-1/7, -1/7, -1/7, -1/7, 0, 0,0]
A[4] = [-1/7,-2/7,0,0,1,-1/7, -1/7, -1/7, -1/7, 0,0]
A[5] = [-1/7,0,-2/7,0,0,1,-1/7, -1/7, -1/7, -1/7, 0]
A[6] = [-1/7,0,0,-2/7,0,0,1,-1/7, -1/7, -1/7, -1/7]
A[7] = [-1/7,0,0,0,-2/7,0,0,1,-1/7, -1/7, -2/7]
A[8] = [-1/7,0,0,0,0,-2/7,0,0,1,-1/7, -3/7]
A[9] = [-1/7,0,0,0,0,0,-2/7,0,0,1, -4/7]
A[10][10] = 1
print(A)

b = np.ones(11)
b[10] = 0
print(b)

[[ 0.57142857 -0.14285714 -0.14285714 -0.14285714 -0.14285714  0.
   0.          0.          0.          0.          0.        ]
 [-0.42857143  1.         -0.14285714 -0.14285714 -0.14285714 -0.14285714
   0.          0.          0.          0.          0.        ]
 [-0.42857143  0.          1.         -0.14285714 -0.14285714 -0.14285714
  -0.14285714  0.          0.          0.          0.        ]
 [-0.42857143  0.          0.          1.         -0.14285714 -0.14285714
  -0.14285714 -0.14285714  0.          0.          0.        ]
 [-0.14285714 -0.28571429  0.          0.          1.         -0.14285714
  -0.14285714 -0.14285714 -0.14285714  0.          0.        ]
 [-0.14285714  0.         -0.28571429  0.          0.          1.
  -0.14285714 -0.14285714 -0.14285714 -0.14285714  0.        ]
 [-0.14285714  0.          0.         -0.28571429  0.          0.
   1.         -0.14285714 -0.14285714 -0.14285714 -0.14285714]
 [-0.14285714  0.          0.          0.         -0.28571429  0.

In [3]:
x = np.linalg.solve(A, b)
print(x)

[17.88160854 17.34461086 16.63636178 15.80113986 14.74432164 13.58562712
 11.67861825  9.95458641  8.42058948  6.89126358  0.        ]
